In [1]:
!git clone https://github.com/AdityaKarn/lane-detection-dataset

Cloning into 'lane-detection-dataset'...
remote: Enumerating objects: 486, done.
remote: Counting objects: 100% (486/486), done.
remote: Compressing objects: 100% (486/486), done.
remote: Total 1083 (delta 0), reused 486 (delta 0), pack-reused 597
Receiving objects: 100% (1083/1083), 112.90 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
%cd lane-detection-dataset

/content/lane-detection-dataset


In [3]:
!pip3 install noise

     |████████████████████████████████| 133kB 7.9MB/s 
  Created wheel for noise: filename=noise-1.2.2-cp36-cp36m-linux_x86_64.whl size=62462 sha256=10cdd132bbbe1d7f71f65c8e4a3a82a9ee59a0b5dd08effecb459659d2fc9d9f
  Stored in directory: /root/.cache/pip/wheels/fd/a3/c1/d36defe6e9f074b25dc0f018eb9f8fdd675a7ef87071ce3821
Successfully built noise


In [4]:
import numpy as np
import cv2

import keras 
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

from sklearn.utils import shuffle
from imgaug import augmenters as iaa

import random
import copy 
import os
import pickle
import matplotlib.pyplot as plt

from image_alteration_v2 import default_alter, draw_points
import multiprocessing


In [5]:
WIDTH = 240
HEIGHT = 135
crop = int(HEIGHT * 0.259525)
cropped_height = HEIGHT - crop
labels_path = 'labels/'
val_labels_path = 'val_labels/'


labels_pahts = os.listdir(labels_path)
val_labels_paths = os.listdir(val_labels_path)


In [6]:
def img_crop(img, rl, ll):  
    img = img[crop:,:,:]
    rl = np.asarray(rl)
    ll = np.asarray(ll)
    rl[:, 1] -= crop
    ll[:, 1] -= crop
    return img, rl, ll

In [7]:
def img_preprocess(img, right_lane, left_lane):
    img, rl, ll = img_crop(img, right_lane, left_lane)
    img = cv2.resize(img, (WIDTH, cropped_height))

    rl_div_x = (rl[:,0]) / WIDTH
    rl_div_y = (rl[:,1]) / cropped_height
    rl = [rl_div_x, rl_div_y]
    rl = np.asarray(rl)
    rl = rl.transpose()

    ll_div_x = (ll[:,0]) / WIDTH
    ll_div_y = (ll[:,1]) / cropped_height
    ll = [ll_div_x, ll_div_y]
    ll = np.asarray(ll)
    ll = ll.transpose()

    # img = cv2.cvtColor(img, cv2.COLOR)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img/255
    return img, rl, ll

In [8]:
def batch_generator(labels_path, batch_size, istraining):
    labels_pahts = os.listdir(labels_path)
    while True:
        batch_img = []
        batch_points = []

        for i in range(batch_size):

            label = pickle.load(open(labels_path + labels_pahts[i], "rb"))
            img, rl, ll = label['img'], label['right_lane'], label['left_lane']

            if istraining:
                img, rl, ll = default_alter(img, rl, ll)


            img, rl, ll = img_preprocess(img, rl, ll)
            # Flatten and concat points
            rl_ravel = np.ravel(rl)
            ll_ravel = np.ravel(ll)
            points = np.concatenate((rl_ravel, ll_ravel))
            
            batch_img.append(img)
            points = points[:24]
            batch_points.append(points)

        batch_img = np.asarray(batch_img)
        batch_points = np.asarray(batch_points)
        # print(batch_points.shape)
        yield (batch_img, batch_points)  

In [9]:
def detection_model():
    model = Sequential()
    model.add(Conv2D(36, (5, 5), strides=(2,2), input_shape=(cropped_height,WIDTH,3), activation='relu'))
    model.add(Conv2D(54, (5, 5), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (5, 5), strides=(2,2), activation='relu'))
    model.add(Conv2D(96, (3, 3), activation='relu'))
    model.add(Conv2D(96, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(232, activation='relu'))
    model.add(Dense(116, activation='relu'))
    model.add(Dense(24))
    
    model.compile(optimizer = Adam(lr=1e-4, decay=0.00000010), loss = 'mse')
    return model

In [10]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://94c81491a8fd.ngrok.io


In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
filepath = "/content/gdrive/My Drive/checkpoints/epochs:{epoch:03d}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, period=100, mode='max')

In [13]:
model = detection_model()

In [14]:
h = model.fit_generator(batch_generator(labels_path, 24, 1), 
                                        steps_per_epoch=20,
                                        epochs=40, 
                                        validation_data=batch_generator(val_labels_path, 24, 1),
                                        validation_steps=10, 
                                        verbose=True, 
                                        shuffle=True,
                                        callbacks=[checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
20/20 [==============================] - 25s 1s/step - loss: 0.1512 - val_loss: 0.0501
Epoch 2/40
20/20 [==============================] - 25s 1s/step - loss: 0.0346 - val_loss: 0.0260
Epoch 3/40
20/20 [==============================] - 25s 1s/step - loss: 0.0230 - val_loss: 0.0189
Epoch 4/40
20/20 [==============================] - 25s 1s/step - loss: 0.0190 - val_loss: 0.0153
Epoch 5/40
20/20 [==============================] - 26s 1s/step - loss: 0.0150 - val_loss: 0.0131
Epoch 6/40
20/20 [==============================] - 26s 1s/step - loss: 0.0121 - val_loss: 0.0117
Epoch 7/40
20/20 [==============================] - 26s 1s/step - loss: 0.0097 - val_loss: 0.0066
Epoch 8/40
20/20 [==============================] - 25s 1s/step - loss: 0.0082 - val_loss: 0.0067
Epoch 9/40
20/20 [==============================] - 25s 1s/step - loss: 0.0074 - val_loss: 0.0058
Epoch 10/40
20/20 [=======================

In [15]:
model.save("model_1.h5")

In [17]:
from google.colab import files
files.download('model_1.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>